# Data Preparation

In [ ]:
import sys
from copy import deepcopy
import warnings
import numpy as np
from numpy import inf, nan
import pandas as pd
import joblib
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
import featuretools as ft

sys.path.append("../../")

from insurance_charges_model.prediction.transformers import DFSTransformer
from insurance_charges_model.prediction.transformers import InfinityToNaNTransformer
from insurance_charges_model.prediction.transformers import IntToFloatTransformer
from insurance_charges_model.prediction.transformers import BooleanTransformer

warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

## Loading the Data

In [ ]:
df = pd.read_csv("../../data/insurance.csv")
df.head()

## Deep Feature Synthesis
We'll be using the featuretools package to do feature engineering.

An EntitySet is an object that we will give to the featuretools package in order to do feature engineering. An entitySet denotes the features of specific "entity" in the real world. In this case, we will work with only one type of entity: "transactions".

In [ ]:
entityset = ft.EntitySet(id="Transactions")
entityset = entityset.entity_from_dataframe(entity_id="Transactions",
                                            dataframe=df,
                                            make_index=True,
                                            index="index")

entityset

In [ ]:
# getting a list of variables associated with the EntitySet we just created
entityset["Transactions"].variables

Now that we have defined an EntitySet for our data, we'll ask the featuretools package to create some features for us. The package defines a set of "primitives" that are able to create new features by processing the features that already exist in the EntitySet.

We are also going to ignore the categorical and boolean features in the dataset because they don't play well with the numerical features.

In [ ]:
feature_dataframe, features = ft.dfs(entityset=entityset,
                                     target_entity="Transactions",
                                     trans_primitives=["add_numeric", "subtract_numeric",
                                                       "multiply_numeric", "divide_numeric",
                                                       "greater_than", "less_than"],
                                     # ignoring some variables
                                     ignore_variables={"Transactions": ["sex", "smoker", "region", "charges"]})

features

In [ ]:
print("We have created {} new features from the original {} features.".format(len(features), len(df.columns) - 2))

The feature_dataframe variable now contains the new features:

In [ ]:
feature_dataframe.head()

## Encode Features using Deep Feature Synthesis

Now we can create a Transformer that we can use later to create the features, given samples of the dataset.

In [ ]:
dfs_transformer = DFSTransformer("Transactions",
                                 trans_primitives=["add_numeric", "subtract_numeric",
                                                   "multiply_numeric", "divide_numeric",
                                                   "greater_than", "less_than"],
                                 ignore_variables={"Transactions": ["sex", "smoker", "region"]})

In [ ]:
# testing the transformer
test_df = pd.DataFrame([[65, "male", 12.5, 0, "yes", "southwest"],
                        [75, "female", 78.770, 1, "no", "southeast"]],
                       columns=["age", "sex", "bmi", "children", "smoker", "region"])

# copying the transformer object in order to fit and test it
dfs_transformer_copy = deepcopy(dfs_transformer)

dfs_transformer_copy.fit(test_df)
new_df = dfs_transformer_copy.transform(test_df)

if len(new_df.columns) != 30:
    raise ValueError("Unexpected number of columns found in the dataframe.")

new_df.head()

## Create Transformer for inf Values

Some of the features created by the featuretools package have a value of 'inf'. We'll create a transformer that maps these values to 0.0 to allow the models to be trained.

In [ ]:
infinity_transformer = InfinityToNaNTransformer()

In [ ]:
# testing the transformer
inpt = [[1.0], [inf], [1.0]]

# copying the transformer object in order to fit and test it
infinity_transformer_copy = deepcopy(infinity_transformer)

infinity_transformer_copy.fit(inpt)
result = infinity_transformer_copy.transform(inpt)

print(result)

In order to handle the NaN values, we'll use a SimpleImputer that will fill in the missing value:

In [ ]:
simple_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
# testing the transformer

# copying the transformer object in order to fit and test it
simple_imputer_copy = deepcopy(simple_imputer)

simple_imputer_copy.fit(result)

test_df = [[np.nan, 2, 3], [4, np.nan, 6], [10, np.nan, 9]]
print(simple_imputer_copy.transform(result))

The SimpleImputer transformer has problems with imputing values that are not floats when using the 'mean' strategy. To fix this, we'll create a transformer that will convert all integer columns into floating point columns:

In [ ]:
int_to_float_transformer = IntToFloatTransformer()

In [ ]:
# test the transformer

# copying the transformer object in order to fit and test it
int_to_float_transformer_copy = deepcopy(int_to_float_transformer)

int_to_float_transformer_copy.fit(result)

test_df = [[2, 3.0], [4, 6.0], [10, 9.0]]
print(int_to_float_transformer_copy.transform(test_df))


Lastly, we'll put the IntToFloatTransformer, DFSTransformer, InfinityToNaNTransformer, and SimpleImputer transformers into a Pipeline so they'll all work together as a unit:

In [ ]:
dfs_pipeline = Pipeline([
    ("dfs_transformer", dfs_transformer),
    ("int_to_float_transformer", int_to_float_transformer),
    ("infinity_transformer", infinity_transformer),
    ("simple_imputer", simple_imputer),
])

In [ ]:
# testing the transformer
test_df = pd.DataFrame([[65, 12.5, 0],
                        [75, 78.770, 1]],
                       columns=["age", "bmi", "children"])

# copying the transformer object in order to fit and test it
dfs_pipeline_copy = deepcopy(dfs_pipeline)

dfs_pipeline_copy.fit(test_df)
new_df = dfs_pipeline_copy.transform(test_df)

if len(new_df[0]) != 30:
    raise ValueError("Unexpected number of columns found in the dataframe.")

## Encode Boolean Features

We'll create a transformer that is able to convert the string in the 'smoker' feature to a boolean value.

In [ ]:
boolean_transformer = BooleanTransformer(true_value="yes", false_value="no")

In [ ]:
# testing the transformer
test_df = pd.DataFrame([["yes"], ["no"], ["yes"]],
                       columns=["smoker"])

# copying the transformer object in order to fit and test it
boolean_transformer_copy = deepcopy(boolean_transformer)

boolean_transformer_copy.fit(test_df)
result = boolean_transformer_copy.transform(test_df)

if (result != np.array([[True], [False], [True]])).all():
    raise ValueError("Unexpected values found in array.")

## Encode Categorical Features

Next, we'll create an encoder that will encode the categorical features. The categorical features that we will encode will be 'sex' and 'region'.

In [ ]:
ordinal_encoder = OrdinalEncoder()

In [ ]:
# testing the transformer
test_df = pd.DataFrame([["southwest"], ["northeast"], ["southwest"]],
                       columns=["region"])

# copying the transformer object in order to fit and test it
ordinal_encoder_copy = deepcopy(ordinal_encoder)

ordinal_encoder_copy.fit(test_df)
result = ordinal_encoder_copy.transform(test_df)

if (result != np.array([[1.0], [0.0], [1.0]])).all():
    raise ValueError("Unexpected values found in array.")

## Create ColumnTransformer

Combining all of the preprocessors into one ColumnTransformer that can be used to preprocess the data.

In [ ]:
column_transformer = ColumnTransformer(
    remainder="passthrough",
    transformers=[
        ("dfs_pipeline", dfs_pipeline, ["age", "sex", "bmi", "children", "smoker", "region"]),
        ("boolean_transformer", boolean_transformer, ["smoker"]),
        ("ordinal_encoder", ordinal_encoder, ["sex", "region"])
    ]
)

In [ ]:
# testing the ColumnTransformer
test_df = pd.DataFrame([[65, "male", 12.5, 0, "yes", "southwest"],
                        [75, "female", 78.770, 1, "no", "southeast"]],
                       columns=["age", "sex", "bmi", "children", "smoker", "region"])

# copying the transformer object in order to fit and test it
column_transformer_copy = deepcopy(column_transformer)

column_transformer_copy.fit(test_df)

result = column_transformer_copy.transform(test_df)

if len(result[0]) != 33:  # expecting 33 features to come out of the ColumnTransformer
    raise ValueError("Unexpected number of columns found in the dataframe.")

## Saving ColumnTransformer

NOTE: the ColumnTransformer object is saved in an UNFITTED state, it will be fitted to the data set later

In [ ]:
joblib.dump(column_transformer, "transformer.joblib")